In [ ]:
!curl -LsSf https://astral.sh/uv/install.sh | sh
!uv pip install python-a2a

In [1]:
# open a terminal and run: python agent1.py
# open another terminal and run: python agent2.py
# open another terminal and run: python agent3.py

# after that, run the following cells

In [1]:
from python_a2a import AgentNetwork, A2AClient, AIAgentRouter

# Create an agent network
network = AgentNetwork(name="Math Assistant Network")

# Add agents to the network
network.add("Sine", "http://localhost:4737")
network.add("Cosine", "http://localhost:4738")
network.add("Tangent", "http://localhost:4739")


llm = A2AClient(endpoint_url="http://localhost:8000")
router = AIAgentRouter(llm_client=llm, agent_network=network)


INFO:python_a2a.client.network:Added agent 'Sine' from URL: http://localhost:4737
INFO:python_a2a.client.network:Added agent 'Cosine' from URL: http://localhost:4738
INFO:python_a2a.client.network:Added agent 'Tangent' from URL: http://localhost:4739


In [ ]:
# Custom routing function to fix the library bug
def route_query_fixed(query, network, llm):
    """Route query by calling LLM directly and parsing response."""
    # Build the routing prompt
    agents_info = []
    for name, agent in network._agents.items():
        agents_info.append(f"- {name}")
    
    prompt = f"""Available agents: {', '.join(network._agents.keys())}

User query: {query}

Which agent should handle this query? Respond with just the agent name."""
    
    # Call the LLM
    response = llm.ask(prompt)
    response_text = str(response).strip()
    
    # Find matching agent name
    for name in network._agents.keys():
        if name.lower() in response_text.lower():
            return name, 0.90
    
    # Fallback to first agent
    return list(network._agents.keys())[0], 0.50

# Route a query to the appropriate agent
query = "What is the cosine of 45 degrees?"
agent_name, confidence = route_query_fixed(query, network, llm)
print(f"Routing to {agent_name} with {confidence:.2f} confidence")

# Get the selected agent and ask the question
agent = network.get_agent(agent_name)
response = agent.ask(query)
print(f"Response: {response}")

In [ ]:
# Route a query to the appropriate agent
query = "Sine of 3.14159"
agent_name, confidence = router.route_query(query)
print(f"Routing to {agent_name} with {confidence:.2f} confidence")

# Get the selected agent and ask the question
agent = network.get_agent(agent_name)
response = agent.ask(query)
print(f"Response: {response}")